In [1]:
import jieba
import re
import pandas as pd
from gensim.models import doc2vec

In [2]:
def purify(s, r_h):
    """
    专门用于清理HTML标签，得出我们想要的处理后的题目
    param s: 待处理字符串
    param r_h: HTML标签集合
    return: 删除HTML标签后的字符串
    """
    for e in r_h:
        s = s.replace(e, ' ')
    return s


In [3]:
def data_proc(data):
    """
    文本前处理：HTML标签删除、斜率截距等关键词
    param data: DataFrame，必含有body和explains列
    return: 处理过HTML、替换一部分关键词后的body和explains列
    """
    qqq = data['body'] + ' [内容分隔] ' + data['explains']
    bd_ex = ''
    for i in qqq:
        i = i + ' [题目分隔] '
        bd_ex += i
    # 将全部题目文本连接成一个长字符串，便于正则直接处理、替换，不做重复事情（可以优化写成同步的）
    pattern_html = re.compile(r'<[^(0-9)][^(<,>,，,$)]*>')
    # 开头第一位是“<”，第二位不能是数字，紧接着匹配任意次的任意字符，但不可以是(<,>,，,$)，最后末尾匹配“>”
    result_html = pattern_html.findall(bd_ex)
    trash = ['< \\\\\\\\frac{a+1}{2} \\\\\\\\\\\\\\\\ x>',
             '< \\\\\\\\frac{m+1}{2} \\\\\\\\\\\\\\\\ x>',
             '< \\\\\\\\frac{m+4}{2} \\\\\\\\\\\\\\\\ x>',
             '<-1\\\\5x-2b>',
             '<-1\\\\\\\\\\\\\\\\5x-2b>',
             '<\\\\\\\\frac{a+2}{3} \\\\\\\\\\\\\\\\ x>',
             '<\\\\\\\\frac{a-1}{4} \\\\\\\\\\\\\\\\ x>',
             '<a+1\\\\\\\\\\\\\\\\ x>',
             '<a+4\\\\\\\\\\\\\\\\ x>',
             '<m+2\\\\\\\\\\\\\\\\ x>',
             '<m-6\\\\\\\\\\\\\\\\ x>',
             '<m-7\\\\\\\\\\\\\\\\ x>',
             '<m-9 \\\\\\\\\\\\\\\\ x>']
    # 这些是匹配到的非HTML标签，不管也罢，最终分词终究被删除
    r_html = sorted(set(result_html))
    res_html = sorted(set(r_html) - set(trash))  # 最终的全部HTML标签
    bd_ex_proc = purify(bd_ex, res_html).split(' [题目分隔] ')  # 按照题目分隔将题目分开
    bd_ex_proc = bd_ex_proc.pop()
    proc_data = pd.Series(bd_ex_proc)

    b_ = []
    e_ = []
    for idx, item in enumerate(proc_data):
        b_e = item.split(' [内容分隔] ')
        b_.append(b_e[0])
        e_.append(b_e[1])
    # 按照【内容分隔】将body、explains分开
    data['body'] = pd.Series(b_)
    data['explains'] = pd.Series(e_)
    return data

In [4]:
data = pd.read_csv('/Users/shen-pc/Desktop/WORK/ITS/KR2/LSD_data/problem_0528.csv')

In [11]:
import time
t1 = time.time()
data_proc(data)
print('处理HTML耗时',(time.time()-t1)/60,'min')

IndexError: list index out of range